<a href="https://colab.research.google.com/github/junekimdev/kdt-autodrive4-team3-week11-12/blob/dahyunk_week11/Decision_Making.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/라벨링

/content/drive/MyDrive/라벨링


In [ ]:
import os
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import time

In [ ]:
ls

all.txt            masked_image/     video20_img_anno/  video22_img_anno/
capture_video.avi  masked_img_anno/  video21_img_anno/


In [ ]:
cd ..

/content/drive/MyDrive/라벨링


In [ ]:
cd video21_img_anno

/content/drive/MyDrive/라벨링/video21_img_anno


In [ ]:
annots = os.listdir("Annotations")
images = os.listdir("Images")
annots.sort()
images.sort()
print(len(annots), len(images))

300 300


In [ ]:
def make_annots_list(path, list_):
  object_detector = []
  for li in list_:
    txt_file = open(path+li, "r")
    lines = txt_file.readlines()
    stack = []
    for line in lines:
      stack.append(list(map(float, line.split())))
    object_detector.append(stack)
    txt_file.close()
  return object_detector

In [ ]:
detector = make_annots_list("Annotations/", annots)

In [ ]:
print(detector[0][60])

IndexError: ignored

In [ ]:
print(416*416//2)

86528


In [ ]:
state = ["Drive_State", "Left_State", "Right_State", "Stop_State"]

In [55]:
def decision_making(img_path, detector, current_state):
  state = ["Drive_State", "Left_State", "Right_State", "Stop_State"]
  anchor_box_min = 1400
  anchor_box_max = 5000
  check_list = [-1] # -1 -> 주행 상태
  bbox = [[]] # 확인용, [] -> 초기화값
  for i in detector:
    cls, cx, cy, w, h = int(i[0]), i[1], i[2], i[3], i[4]
    ratio = h/w
    if cls != 5:
      ratio_min = 1.0
      ratio_max = 2.3
      sqare = int(w*173056*h)
    elif cls == 5:
      sqare = int(w*129792*h)
      ratio_min = 1.8
      ratio_max = 3.2
    weight = int(w*416)
    if ratio_min < ratio and ratio_max > ratio:
      if weight > 20 and weight < 60:
        if anchor_box_min > sqare or anchor_box_max < sqare:
          continue
        else:
          anchor_box_min = sqare
          print("밑변은 %f" % weight)
          print("높이는", h*416)
          print("넓이는", sqare)
          print("비율은", h/w)
          bbox.pop()
          bbox.append([cx,cy,w,h])
          check_list.pop()
          check_list.append(cls)
  img = cv2.imread(img_path)
    ########## 확인용 ################
  if bbox[0] != []:
    x_min = int((bbox[0][0]-bbox[0][2]/2) * 416)
    x_max = int((bbox[0][0]+bbox[0][2]/2) * 416)
    y_min = int((bbox[0][1]-bbox[0][3]/2) * 416)
    y_max = int((bbox[0][1]+bbox[0][3]/2) * 416)
    img = cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0,255,0), 1)
    ###############################
  cv2_imshow(img)
    
  if check_list[-1] == -1:
    if current_state == "Drive_State":
      return current_state
    else:
      # print(current_state+" -> "+state[0])
      return state[0]
  elif check_list[-1] == 0:
    # print("Left State")
    if current_state == "Left_State":
      return current_state
    else:
      # print(current_state+" -> " +state[1])
      return state[1]
  elif check_list[-1] == 1:
      # print("Left State")
    if current_state == "Right_State":
      return current_state
    else:
      # print(current_state+" -> " +state[2])
      return state[2]
  elif check_list[-1] == 2 or check_list[-1] == 3:
      # print("Stop State")
    if current_state == "Stop_State":
      time.sleep(5)
      return state[0]
    else:
      # print(current_state+" -> " +state[3])
      return state[3]
  elif check_list[-1] == 5:
      # print("Traffic_light Detection")
      
      # lower_blue = (110,100,100)
      # upper_blue = (125, 255, 255)
      lower_red = (160, 50, 50)
      upper_blue = (180,255,255)
      cx, cy, w, h = bbox[0][0], bbox[0][1], bbox[0][2], bbox[0][3]
      img = cv2.imread(img_path)
      roi_img = img[int((cy-h/2)*416):int((cy+h/2)*416),int((cx-w/4)*416):int((cx+w/4)*416)]
      new_img = cv2.cvtColor(roi_img, cv2.COLOR_BGR2HSV)
      hsv_img = cv2.cvtColor(new_img, cv2.COLOR_BGR2HSV)

      # mask  = cv2.inRange(hsv_img, lower_blue, upper_blue)
      mask_red = cv2.inRange(hsv_img, lower_red, upper_blue)
      # mask_img = cv2.bitwise_and(new_img, new_img, mask = mask)
      mask_img_red = cv2.bitwise_and(new_img, new_img, mask = mask_red)
      # cv2_imshow(mask_img_red)
      gray_img = cv2.cvtColor(mask_img_red, cv2.COLOR_BGR2GRAY)
      ret ,th_img = cv2.threshold(gray_img, 1, 255, cv2.THRESH_BINARY_INV)
      row = int(h*416*0.3)
      col = len(th_img[0])
      sqare = len(th_img)*col
      count1 = 0
      count2 = 0
      count3 = 0
      print("roi1")
      cv2_imshow(th_img[0:row,0:col])
      print("roi2")
      cv2_imshow(th_img[int(0.35*h*416):int(0.35*h*416)+row, 0: col])
      print("roi3")
      cv2_imshow(th_img[int(0.65*h*416):int(0.65*h*416)+row, 0: col])
      for i in range(row):
        for j in range(col):
          if th_img[i][j] == 0:
            count1 += 1
          if th_img[i+int(0.35*h*416)][j] == 0:
            count2 += 1
          if th_img[i+int(0.65*h*416)][j] == 0:
            count3 +=1
      decision1 = int(count1/sqare*1000)
      decision2 = int(count2/sqare*1000)
      decision3 = int(count3/sqare*1000)
      print(decision1, decision2, decision3)
      min_, max_ = min(decision1,decision3), max(decision1, decision3)


      if decision2 > 100:
        print("Yellow")
      elif max_ > 30 and max_ < 60:
        print("Red")
      elif max_ > 100:
        print("Green")
      else:
        print("Turn Off")

      if min_ == 0 and max_ < 15:
        if current_state == "Stop_State":
          return current_state
        else:
          # print(current_state + " -> " + state[3])
          return state[3]
      elif min_ == 0:
        # print("Drive_State")
        if current_state == "Drive_State":
          return current_state
        else:
          # print(current_state + " -> " + state[0])
          return state[0]
      else:
        # print("Stop_State")
        if current_state == "Stop_State":
          return current_state
        else:
          # print(current_state + " -> " + state[3])
          return state[3]

    
    # print("넓이는 %d" % sqare)
    

In [ ]:
detector[0][0]

[3.0, 0.944259, 0.497942, 0.107023, 0.296296]

In [ ]:
num = 0
stack = ["Stop_State"]
img_path = "Images/"
for i, j in zip(images, detector):
  stack.append(decision_making(img_path+i, j, stack[-1]))
  num += 1
  # if num == 100:
  #   break